In [5]:
import dataclasses

import jax
import jax.numpy as jnp

from jax.experimental.pallas.ops.gpu.attention import mha

In [7]:
@dataclasses.dataclass(frozen=True, slots=True)
class BlockSizes:
  block_q: int
  block_k: int

In [8]:
rng = jax.random.PRNGKey(0)

max_seq_len = 64
B, T, nH, C = 32, 15, 8, 128

q = jax.random.normal(rng, (B, T, nH, C))
k = jax.random.normal(rng, (B, T, nH, C))
v = jax.random.normal(rng, (B, T, nH, C))

pad_width = ((0, 0),  # no padding on the first dimension
             (0, max_seq_len-15),
             (0, 0),
             (0, 0))  # pad two zeros on the right side of the second dimension

q = jnp.pad(q, pad_width, mode='constant', constant_values=0)
k = jnp.pad(k, pad_width, mode='constant', constant_values=0)
v = jnp.pad(v, pad_width, mode='constant', constant_values=0)

block_sizes = BlockSizes(block_q=T, block_k=T)

y = mha(q, k, v, block_sizes=block_sizes, segment_ids=None, causal=True)

mha.__str__

XlaRuntimeError: UNKNOWN: -:0:0: error: unknown attribute code: 22
-:0:0: note: in bytecode version 6 produced by: StableHLO_v1.8.3
